In [5]:
import pandas as pd
import numpy as np
from datetime import datetime

In [6]:
df = pd.read_excel('data/fy-2076-77.xlsx', header=1)

In [8]:
def subheading_fix(x):
    if pd.notna(x) and type(x) == datetime and " " in str(x):
        return f'{x.year}.{x.month}.{x.day}'
    return x
df = df[df.iloc[:, 0] != "Heading"]
df = df[df.iloc[:, 3] != 'SAARC']
df = df[df.iloc[:, 0] != 1]
df.columns = ["Heading","Sub-heading","Description of goods","Import Duty SAARC", "Import Duty General"]
df['Sub-heading'] = df['Sub-heading'].apply(subheading_fix)

print(df)

df["Description of goods"] = df["Description of goods"].apply(lambda x: " ".join(x.split()))
df.reset_index(drop=True, inplace=True)

    Heading Sub-heading                     Description of goods  \
3     87.01         NaN  Tractors (other than tractors of 87.09)   
4       NaN         NaN                   -Single axle tractors:   
5       NaN  8701.10.10              --Power tiller, mini tiller   
6       NaN  8701.10.90                                  --Other   
7       NaN         NaN       -Road tractors for semi-trailers::   
..      ...         ...                                      ...   
183     NaN  8716.40.00        -Other trailers and semi-trailers   
184     NaN  8716.80.00                        - Other  vehicles   
185     NaN         NaN                                  -Parts:   
186     NaN  8716.90.10              --Wheels of woods for carts   
187     NaN  8716.90.90                                  --Other   

    Import Duty SAARC Import Duty General  
3                 NaN                 NaN  
4                 NaN                 NaN  
5                   1                   1  
6      

In [9]:
df['Heading'] = np.NaN
main_descriptors = df[~df['Description of goods'].str.startswith('-')].index
for i, ind in enumerate(main_descriptors):
    df.loc[ind, 'Heading'] = f'87.{"0" if i<9 else ""}{i+1}'

/tmp/ipykernel_13644/1324436002.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '87.01' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[ind, 'Heading'] = f'87.{"0" if i<9 else ""}{i+1}'


In [10]:
def count_hyphens_in_prefix(s):
    count = 0
    for char in s:
        if char == '-':
            count += 1
        else:
            break  # Exit the loop if a non-hyphen character is encountered
    return count

for i, row in df[::-1].iterrows():
    if not pd.notna(row['Sub-heading']):
        if pd.notna(row['Heading']):
            df.loc[i, 'Sub-heading'] = row['Heading'].replace('.', '')
        else:
            dashes = count_hyphens_in_prefix(row['Description of goods'])
            prev_code = df.loc[i+1]['Sub-heading']
            if type(prev_code) == str:
                if prev_code.endswith('.00'):
                    prev_code = prev_code[:len(prev_code)-3]
                if prev_code.endswith('0'):
                    new_code = '.'.join(prev_code.split('.')[:-1])
                else:
                    new_code = prev_code[:len(prev_code)-1]+'0'
                df.loc[i, 'Sub-heading'] = new_code

df['Heading'] = df['Heading'].ffill()

In [11]:
df['Description of goods'] = df['Description of goods'].str.lstrip('-')
df = df[['Sub-heading', 'Description of goods', 'Import Duty SAARC', 'Import Duty General']]

In [12]:
df.to_csv('data/fy-2076-77.csv', index=False)